## Convolution
A floating window will scan the image and using convolution, will "convert" the data of the image inside the small window into a value. Than, it will move the window and repeat, until the whole image is convoluted into a smaller array of values.

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

NAME = "Cats-vs-dog-cnn-64x2-{}".format(int(time.time()))  # name the model with unique name

# Constants
CONVOLUTION_WINDOW = (3, 3)  # define a convolution window
LAYER_SIZE = 128  # number of nodes
EPOCHS = 7  

tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)  # utilizes 30%
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))  # uses the configuration


x = pickle.load(open("x.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

# 1st thing we need to do is to normalize the data, for imagry data the easiest way to do that is to divide in the maximum value
x = x/255.0

model = Sequential()  # create a sequential model

model.add(Conv2D(LAYER_SIZE, CONVOLUTION_WINDOW, input_shape=x.shape[1:])) # 64 is the number of nodes
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(LAYER_SIZE, CONVOLUTION_WINDOW, input_shape=x.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(LAYER_SIZE))  # add a Dense layer
model.add(Activation('relu'))

model.add(Dense(1))  # Add output layer with bool value (dog/cat)
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(x, y, batch_size=32, epochs=EPOCHS, validation_split=0.1, callbacks=[tensorboard])  # pass 32 at a time, 10% validation split

Train on 22451 samples, validate on 2495 samples
Epoch 1/7
22451/22451 [==============================] - 434s 19ms/step - loss: 0.6196 - acc: 0.6526 - val_loss: 0.5628 - val_acc: 0.7174
Epoch 2/7
22451/22451 [==============================] - 405s 18ms/step - loss: 0.5258 - acc: 0.7413 - val_loss: 0.4911 - val_acc: 0.7679
Epoch 3/7
22451/22451 [==============================] - 414s 18ms/step - loss: 0.4716 - acc: 0.7752 - val_loss: 0.4856 - val_acc: 0.7723
Epoch 4/7
22451/22451 [==============================] - 412s 18ms/step - loss: 0.4237 - acc: 0.8026 - val_loss: 0.4572 - val_acc: 0.7964
Epoch 5/7
22451/22451 [==============================] - 563s 25ms/step - loss: 0.3765 - acc: 0.8292 - val_loss: 0.4669 - val_acc: 0.7840
Epoch 6/7
22451/22451 [==============================] - 553s 25ms/step - loss: 0.3202 - acc: 0.8594 - val_loss: 0.4770 - val_acc: 0.7932
Epoch 7/7
22451/22451 [==============================] - 343s 15ms/step - loss: 0.2537 - acc: 0.8928 - val_loss: 0.5407 - v

In [12]:
model.save('dogs_cats_detector.model')